<a href="https://colab.research.google.com/github/eridanletalis/GB_nn/blob/main/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D1%83%D1%82%D0%B2%D0%B5%D1%80%D0%B6%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D1%80%D0%BE_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# Проверим, что ГПУ включился
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.0577504720000093
GPU (s):
0.04052248600000041
GPU speedup over CPU: 75x


In [4]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# сразу подключим оптимизаторы, слои и прочее
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.models import Model
import tensorflow

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

Загружаем исходные данные

In [6]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [8]:
y_train_labels = to_categorical(train_labels)
y_test_labels = to_categorical(test_labels)

In [9]:
train_images = train_images / 255.0

test_images = test_images / 255.0

#Теперь поработаем с тестами разных оптимизаторов для подобранной сети

In [10]:
import pandas as pd
from datetime import datetime
from tensorflow.keras import initializers

In [11]:
columns_1 = ("number", "adam", "sgd", "rmsprop", "adagrad")
models_loss = pd.DataFrame(columns=columns_1)
models_acc = pd.DataFrame(columns=columns_1)

In [12]:
optimizers = ["adam", "sgd", "rmsprop", "adagrad"]

## Выполняем эксперимент на определение лучшего оптимизатора
Эксперимент будет выпоняться 15 раз с разными seed для усреднения при разных начальных состояниях. 
Записываем loss, acc.

Далее результат будет усреднён и выбрана модель с наилучшей комбинацией loss-acc. Учитываться будут средние значения и дисперсии.

In [13]:
# Количество слоёв выбиралось исходя из уменьшения вдвое числа входов
for i in range(1,51):
  for opt in optimizers:
    print(f"Выполняется проход {i}, оптимизатор {opt}")
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(392, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, 
                                                                  stddev=0.15, 
                                      seed=int(datetime.now().timestamp()))),
        keras.layers.Dense(196, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, 
                                                                  stddev=0.15, 
                                      seed=int(datetime.now().timestamp()))),
        keras.layers.Dense(98, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, 
                                                                  stddev=0.15, 
                                      seed=int(datetime.now().timestamp()))),
        keras.layers.Dense(49, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, 
                                                                  stddev=0.15, 
                                      seed=int(datetime.now().timestamp()))),
        keras.layers.Dense(10,
                           kernel_initializer=initializers.RandomNormal(mean=0.0, 
                                                                  stddev=0.15, 
                                      seed=int(datetime.now().timestamp())))
    ])

    model.compile(optimizer=opt,
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(train_images, y_train_labels, epochs=18, batch_size=100, verbose=0)

    test_loss, test_acc = model.evaluate(test_images,  y_test_labels, verbose=2)
    models_loss.loc[i, opt] = test_loss
    models_acc.loc[i, opt] = test_acc

Выполняется проход 1, оптимизатор adam
313/313 - 1s - loss: 0.3656 - accuracy: 0.8858
Выполняется проход 1, оптимизатор sgd
313/313 - 1s - loss: 0.3992 - accuracy: 0.8606
Выполняется проход 1, оптимизатор rmsprop
313/313 - 1s - loss: 0.5164 - accuracy: 0.8833
Выполняется проход 1, оптимизатор adagrad
313/313 - 1s - loss: 0.4644 - accuracy: 0.8350
Выполняется проход 2, оптимизатор adam
313/313 - 1s - loss: 0.3821 - accuracy: 0.8854
Выполняется проход 2, оптимизатор sgd
313/313 - 1s - loss: 0.3919 - accuracy: 0.8604
Выполняется проход 2, оптимизатор rmsprop
313/313 - 1s - loss: 0.5303 - accuracy: 0.8789
Выполняется проход 2, оптимизатор adagrad
313/313 - 1s - loss: 0.4815 - accuracy: 0.8345
Выполняется проход 3, оптимизатор adam
313/313 - 1s - loss: 0.3704 - accuracy: 0.8931
Выполняется проход 3, оптимизатор sgd
313/313 - 1s - loss: 0.3850 - accuracy: 0.8610
Выполняется проход 3, оптимизатор rmsprop
313/313 - 1s - loss: 0.5282 - accuracy: 0.8698
Выполняется проход 3, оптимизатор adagrad


In [14]:
models_loss.astype(np.float64).describe()


,number,adam,sgd,rmsprop,adagrad
count,0.0,50.000000,50.000000,50.000000,50.000000
mean,NaN,0.373448,0.394177,0.549075,0.476162
std,NaN,0.013989,0.009156,0.070741,0.005624
min,NaN,0.340821,0.380063,0.440174,0.464412
25%,NaN,0.365651,0.389728,0.507449,0.471933
50%,NaN,0.373431,0.392127,0.533559,0.476542
75%,NaN,0.381391,0.397973,0.591290,0.479226
max,NaN,0.410880,0.440758,0.878803,0.489832


In [15]:
models_acc.astype(np.float64).describe()

,number,adam,sgd,rmsprop,adagrad
count,0.0,50.000000,50.000000,50.000000,50.000000
mean,NaN,0.887610,0.859630,0.877450,0.832972
std,NaN,0.003736,0.003994,0.009565,0.001996
min,NaN,0.871700,0.838200,0.830500,0.828200
25%,NaN,0.885300,0.858225,0.874100,0.831750
50%,NaN,0.887850,0.860050,0.879850,0.832850
75%,NaN,0.890400,0.861650,0.883975,0.834475
max,NaN,0.893700,0.865200,0.887600,0.836800


#Проверка с одним seed

In [16]:
columns_1 = ("number", "adam", "sgd", "rmsprop", "adagrad")
models_loss = pd.DataFrame(columns=columns_1)
models_acc = pd.DataFrame(columns=columns_1)

In [17]:
optimizers = ["adam", "sgd", "rmsprop", "adagrad"]

## Выполняем эксперимент на определение лучшего оптимизатора
Эксперимент будет выпоняться 15 раз с разными seed для усреднения при разных начальных состояниях. 
Записываем loss, acc.

Далее результат будет усреднён и выбрана модель с наилучшей комбинацией loss-acc. Учитываться будут средние значения и дисперсии.

In [ ]:
# Количество слоёв выбиралось исходя из уменьшения вдвое числа входов
for i in range(1,51):
  for opt in optimizers:
    print(f"Выполняется проход {i}, оптимизатор {opt}")
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(392, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.15,seed=int(42))),
        keras.layers.Dense(196, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.15,seed=int(42))),
        keras.layers.Dense(98, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.15,seed=int(42))),
        keras.layers.Dense(49, activation='relu',
                           kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.15,seed=int(42))),
        keras.layers.Dense(10,
                           kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.15,seed=int(42)))
    ])

    model.compile(optimizer=opt,
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(train_images, y_train_labels, epochs=18, batch_size=100, verbose=0)

    test_loss, test_acc = model.evaluate(test_images,  y_test_labels, verbose=2)
    models_loss.loc[i, opt] = test_loss
    models_acc.loc[i, opt] = test_acc

In [29]:
models_loss.astype(np.float64).describe()


,number,adam,sgd,rmsprop,adagrad
count,0.0,50.000000,50.000000,50.000000,50.000000
mean,NaN,0.374641,0.391096,0.550405,0.477420
std,NaN,0.016966,0.008053,0.057247,0.001243
min,NaN,0.338099,0.380241,0.442757,0.474607
25%,NaN,0.364294,0.385634,0.517282,0.476771
50%,NaN,0.373714,0.389420,0.539066,0.477368
75%,NaN,0.385032,0.393431,0.574520,0.478059
max,NaN,0.417933,0.417023,0.670928,0.480963


In [30]:
models_acc.astype(np.float64).describe()

,number,adam,sgd,rmsprop,adagrad
count,0.0,50.000000,50.000000,50.000000,50.000000
mean,NaN,0.886936,0.859236,0.878078,0.833742
std,NaN,0.004751,0.003493,0.008280,0.001203
min,NaN,0.872800,0.847600,0.851600,0.830900
25%,NaN,0.884650,0.858325,0.873875,0.832900
50%,NaN,0.887650,0.859900,0.880050,0.833650
75%,NaN,0.890275,0.861575,0.883350,0.834775
max,NaN,0.893600,0.864300,0.890400,0.836000
